In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

f:\school\cs-thesis-writing\course-platform\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [4]:
df = tfds.as_dataframe(ratings.take(10))
df

,movie_title,user_id
0,"b""One Flew Over the Cuckoo's Nest (1975)""",b'138'
1,b'Strictly Ballroom (1992)',b'92'
2,"b'Very Brady Sequel, A (1996)'",b'301'
3,b'Pulp Fiction (1994)',b'60'
4,b'Scream 2 (1997)',b'197'
5,b'Crash (1996)',b'601'
6,b'Aladdin (1992)',b'710'
7,b'True Romance (1993)',b'833'
8,b'Bob Roberts (1992)',b'916'
9,b'Starship Troopers (1997)',b'940'


In [5]:
movies

<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [6]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

## Define a model


In [7]:
class MovieLensModel(tfrs.Model):
    def __init__(self,
                 user_model: tf.keras.Model,
                 movie_model: tf.keras.Model,
                 task: tfrs.tasks.Retrieval):

        super().__init__()

        self.user_model = user_model
        self.movie_model = movie_model

        self.task = task
    

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])
        print("COMPUTE LOSS: ")
        print(features)
        return self.task(user_embeddings, movie_embeddings)

In [8]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
    )
)

In [9]:
df = tfds.as_dataframe(ratings.take(10))
df

,movie_title,user_id
0,"b""One Flew Over the Cuckoo's Nest (1975)""",b'138'
1,b'Strictly Ballroom (1992)',b'92'
2,"b'Very Brady Sequel, A (1996)'",b'301'
3,b'Pulp Fiction (1994)',b'60'
4,b'Scream 2 (1997)',b'197'
5,b'Crash (1996)',b'601'
6,b'Aladdin (1992)',b'710'
7,b'True Romance (1993)',b'833'
8,b'Bob Roberts (1992)',b'916'
9,b'Starship Troopers (1997)',b'940'


In [10]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(4096), epochs=5)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/5
COMPUTE LOSS: 
{'movie_title': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=string>, 'user_id': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=string>}
COMPUTE LOSS: 
{'movie_title': <tf.Tensor 'IteratorGetNext:0' shape=(None,) dtype=string>, 'user_id': <tf.Tensor 'IteratorGetNext:1' shape=(None,) dtype=string>}
25/25 [==============================] - 5s 171ms/step - factorized_top_k/top_1_categorical_accuracy: 1.4000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0016 - factorized_top_k/top_10_categorical_accuracy: 0.0049 - factorized_top_k/top_50_categorical_accuracy: 0.0454 - factorized_top_k/top_100_categorical_accuracy: 0.1023 - loss: 33088.3559 - regularization_loss: 0.0000e+00 - total_loss: 33088.3559
Epoch 2/5
25/25 [==============================] - 4s 175ms/step - factorized_top_k/top_1_categorical_accuracy: 1.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0053 - factorized_top_k/top_10_categorical_accuracy: 0.0146 - factorized_top_k/top_